In [8]:
import xarray
from distributed import Client
import sys
sys.path.insert(1, '../../heatwave_analysis_package/src')
import hdp
import cftime
import numpy as np

client = Client('localhost:8786')
client

<Client: 'tcp://146.6.192.149:8786' processes=20 threads=40, memory=0.91 TiB>

In [9]:
INPUT_DIR = "/projects/dgs/persad_research/SIMULATION_DATA/ZARR/LENS1/SIM_VARIABLES/"
OUTPUT_DIR = "/projects/dgs/persad_research/SIMULATION_DATA/ZARR/LENS1/HEAT_OUTPUTS/"

pi_time_start = cftime.DatetimeNoLeap(1920, 1, 1, 0, 0, 0, 0, has_year_zero=True)
pi_time_end = cftime.DatetimeNoLeap(2200, 12, 31, 0, 0, 0, 0, has_year_zero=True)
tmax_pi_path = f"{INPUT_DIR}preindustrial_day_TREFHTMX.zarr"

tmax_pi_ds = xarray.open_zarr(tmax_pi_path)["TREFHTMX"]
tmax_pi_ds = tmax_pi_ds.sel(time=slice(pi_time_start, pi_time_end))

print("Computing threshold...", end=" ")
tmax_threshold = hdp.compute_threshold(tmax_pi_ds, np.array([0.9, 0.95]), temp_path=tmax_pi_path)
tmax_threshold.chunk(dict(percentile=-1, day=-1, lat=-1, lon=-1)).to_zarr(f"{OUTPUT_DIR}CESM1_LE_TREFHTMX_THRESHOLD.zarr", consolidated=True, mode="w")
print("Done.")

tmax_all_path = f"{INPUT_DIR}all_day_TREFHTMX.zarr"
tmax_xaer_path = f"{INPUT_DIR}xaer_day_TREFHTMX.zarr"
tmax_xghg_path = f"{INPUT_DIR}xghg_day_TREFHTMX.zarr"

tmax_all_ds = xarray.open_zarr(tmax_all_path)["TREFHTMX"]
tmax_xaer_ds = xarray.open_zarr(tmax_xaer_path)["TREFHTMX"]
tmax_xghg_ds = xarray.open_zarr(tmax_xghg_path)["TREFHTMX"]

all_hw_ds = []
xaer_hw_ds = []
xghg_hw_ds = []

print("Computing metrics for members:", end=" ")
for member in tmax_xghg_ds.member:
    all_hw_ds.append(hdp.compute_heatwave_metrics(tmax_all_ds.sel(member=member).compute(), tmax_threshold.sel(percentile=[0.9, 0.95], method="nearest")))
    xaer_hw_ds.append(hdp.compute_heatwave_metrics(tmax_xaer_ds.sel(member=member).compute(), tmax_threshold.sel(percentile=[0.9, 0.95], method="nearest")))
    xghg_hw_ds.append(hdp.compute_heatwave_metrics(tmax_xghg_ds.sel(member=member).compute(), tmax_threshold.sel(percentile=[0.9, 0.95], method="nearest")))
    print(member, end=", ")
print("Done.")

print("Concatenating and saving out...")
all_hw_ds_ensemble = xarray.concat(all_hw_ds, dim="member")
xaer_hw_ds_ensemble = xarray.concat(xaer_hw_ds, dim="member")
xghg_hw_ds_ensemble = xarray.concat(xghg_hw_ds, dim="member")

all_hw_ds_ensemble.to_zarr(f"{OUTPUT_DIR}CESM1_LE_all_TREFHTMX_HW_METRICS.zarr", consolidated=True, mode="w")
xaer_hw_ds_ensemble.to_zarr(f"{OUTPUT_DIR}CESM1_LE_xaer_TREFHTMX_HW_METRICS.zarr", consolidated=True, mode="w")
xghg_hw_ds_ensemble.to_zarr(f"{OUTPUT_DIR}CESM1_LE_xghg_TREFHTMX_HW_METRICS.zarr", consolidated=True, mode="w")

tmax_pi_ds = xarray.open_zarr(tmax_pi_path)["TREFHTMX"]
tmax_pi_ds = tmax_pi_ds.sel(time=slice(pi_time_start, pi_time_end))

tmax_threshold = xarray.open_zarr(f"{OUTPUT_DIR}CESM1_LE_TREFHTMX_THRESHOLD.zarr")

print("Computing and outputting preindustrial control metrics...", end=" ")
tmax_pi_hw = hdp.compute_heatwave_metrics(tmax_pi_ds.compute(), tmax_threshold.sel(percentile=[0.9, 0.95], method="nearest"))
tmax_pi_hw.to_zarr(f"{OUTPUT_DIR}CESM1_LE_pic_TREFHTMX_HW_METRICS.zarr", consolidated=True, mode="w")
print("Done.")

Computing threshold... 

/home/jupyterhub/miniconda-persad/envs/lab/lib/python3.12/site-packages/distributed/client.py:3164: UserWarning: Sending large graph of size 153.99 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Done.
Computing metrics for members: 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, <xarray.DataArray 'member' ()> Size: 12B
array('001', dtype='<U3')
Coordinates:
    member   <U3 12B '001', 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, <xarray.DataArray 'member' ()> Size: 12B
array('002', dtype='<U3')
Coordinates:
    member   <U3 12B '002', 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, <xarray.DataArray 'member' ()> Size: 12B
array('003', dtype='<U3')
Coordinates:
    member   <U3 12B '003', 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, <xarray.DataArray 'member' ()> Size: 12B
array('004', dtype='<U3')
Coordinates:
    member   <U3 12B '004', 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, <xarray.DataArray 'member' ()> Size: 12B
array('005', dtype='<U3')
Coordinates:
    member   <U3 12B '005', 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, <xarray.DataArray 'member' ()> Size: 12B
array('006', dtype='<U3')
Coordinates:
    member   <U3 12B '006', 0.9, 0.95, 0.9, 0.95, 0.9, 0.95, <xarray.DataArray 'member' ()> Size: 12B
array('007', dtype='<U3')
Coordinates:
    